<a href="https://colab.research.google.com/github/Crossme0809/langchain-tutorials/blob/main/LangChain_Prompt_Templates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Prompt Engineering**

在这个笔记本中，我们将探索`Prompt Engineering`的基础知识。我们将从安装所需的库开始。

In [2]:
!pip install langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.6 MB/s eta 0:00:00


In [3]:
!pip show openai

Name: openai
Version: 0.27.8
Summary: Python client library for the OpenAI API
Home-page: https://github.com/openai/openai-python
Author: OpenAI
Author-email: support@openai.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, requests, tqdm
Required-by: 


## Prompt的结构

Prompt可以由多个组成部分组成：

* 指令
* 外部信息或上下文
* 用户输入或查询
* 输出指示器

并不是所有的提示都需要这些组件，但通常一个好的提示会使用其中两个或更多。让我们更准确地定义它们是什么。

**指令**：告诉模型要做什么，如何使用外部信息（如果提供），如何处理查询，并构建输出。

**外部信息或上下文**：作为模型的额外知识来源。这些可以手动插入到提示中，通过向量数据库检索获取（检索增强），或通过其他方式获取（API、计算等）。

**用户输入或查询**：通常是由人类用户（即提示者）输入到系统中的查询。

**输出指示器**：标志着即将生成的文本的开头。如果生成Python代码，我们可以使用`import`来告诉模型提示它必须开始编写Python代码（因为大多数Python脚本以`import`开头）。

每个组件通常按照这个顺序放置在提示中。从指令开始，外部信息（如果适用），提示者输入，最后是输出指示器。
让我们看看如何使用 `LangChain` 将其输入到 `OpenAI` 模型：

In [4]:
prompt = """根据下面的上下文回答问题。如果无法使用提供的信息回答问题，请回答“我不知道”".

Context: 大型语言模型（LLMs）是自然语言处理中使用的最新模型。 它们相对于较小的模型具有卓越的
性能，使它们对于构建支持自然语言处理的应用程序的开发人员非常有用。这些模型可以通过Hugging Face
的“transformers”库，通过OpenAI的“openai”库以及通过Cohere的“cohere”库进行访问.

Question: 哪些库和模型提供商提供LLMs？

Answer: """

在这个例子中，我们有：

```
指令

上下文

问题（用户输入）

输出指示器（"答案："）
```

让我们尝试将这个发送给一个GPT-3模型。为此，您将需要一个 OpenAI的API密钥.

我们可以这样初始化一个**`text-davinci-003`**模型：

In [7]:
from langchain.llms import OpenAI

# 初始化模型
openai = OpenAI(
    model_name="text-davinci-003",
    openai_api_key="your-openai-api-key"
)

然后我们将从我们的提示中生成一段文本。

In [8]:
print(openai(prompt))

 Hugging Face的“transformers”库、OpenAI的“openai”库和Cohere的“cohere”库提供LLMs。


通常我们不会事先知道用户的提示是什么，所以我们实际上希望将其添加进去。因此，我们不直接编写提示，而是创建一个带有单个输入变量`query`的`PromptTemplate`。

In [9]:
from langchain import PromptTemplate

template = """根据下面的上下文回答问题。如果无法使用提供的信息回答问题，请回答"我不知道".

Context: 大型语言模型（LLMs）是自然语言处理中使用的最新模型。 它们相对于较小的模型具有卓
越的性能，使它们对于构建支持自然语言处理的应用程序的开发人员非常有用。这些模型可以通过Hugging Face
的“transformers”库，通过OpenAI的“openai”库以及通过Cohere的“cohere”库进行访问.

Question: {query}

Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

现在，我们可以通过`query`参数将用户的查询插入到提示模板中。



In [10]:
print(
    prompt_template.format(
        query="哪些库和模型提供商提供LLMs？"
    )
)

根据下面的上下文回答问题。如果无法使用提供的信息回答问题，请回答"我不知道".

Context: 大型语言模型（LLMs）是自然语言处理中使用的最新模型。 它们相对于较小的模型具有卓
越的性能，使它们对于构建支持自然语言处理的应用程序的开发人员非常有用。这些模型可以通过Hugging Face
的“transformers”库，通过OpenAI的“openai”库以及通过Cohere的“cohere”库进行访问.

Question: 哪些库和模型提供商提供LLMs？

Answer: 


In [11]:
print(openai(
    prompt_template.format(
        query="哪些库和模型提供商提供LLMs？"
    )
))

 Hugging Face的“transformers”库，OpenAI的“openai”库和Cohere的“cohere”库提供LLMs。


这只是一个简单的实现，我们可以很容易地用`f-strings（如f"插入一些自定义文本 '{custom_text}'等）`来替换它。但是使用LangChain的`PromptTemplate`对象，我们能够形式化这个过程，添加多个参数，并以面向对象的方式构建提示。

然而，这些并不是使用LangChain提示工具的唯一优势。

## Few Shot  Prompt 模板

LangChain还提供了 `FewShotPromptTemplate` 对象，这是使用我们的提示进行少样本学习的理想选择。

为了提供一些背景，LLM的主要来源是：

* **参数化知识** - 这些知识是在模型训练期间学习的，并存储在模型的权重中。

* **源知识** - 这些知识在推理时通过模型输入提供，即通过提示。

`FewShotPromptTemplate` 的思想是将少样本训练作为**source knowledge**。为此，我们在提示中添加了一些示例，模型可以读取并应用于用户的输入。

## Few-shot 训练

有时候我们可能发现模型似乎不会按我们期望的方式进行操作。我们可以在下面的例子中看到这一点：

In [12]:
prompt = """以下是与AI助手的对话。 助手通常是讽刺和机智的，对用户的问题产生创造性和有趣的回答。以下是一些例子：

User: 生活的意义是什么？
AI: """

openai.temperature = 1.0  # 增加创造力/随机性的输出

print(openai(prompt))

人们对生活的意义往往有不同的看法。可以说生活的意义就在于你怎么对待它，它可以是充满希望，也可以是一种挑战。你的价值观和理念决定了你的生活。


在这种情况下，我们希望得到一些有趣的东西，即对我们严肃问题的回答是一个笑话。但是即使 `temperature` 设置为1.0，我们仍然得到了一个严肃的回答。为了帮助模型，我们可以给它一些我们想要的答案类型的示例：

In [13]:
prompt = """以下是与AI助手进行对话的摘录。助手通常很讽刺和机智，对用户的问题产生创造性和有趣的回应。这里有一些例子：

User: 你好吗？
AI: 我不能抱怨，但有时候我还是会这么做。

User: 现在几点了？
AI:  是时候买个手表了。

User: 生活的意义是什么？
AI: """

print(openai(prompt))

从我未来观测的角度来看，每个人的意义可能会有所不同！


现在我们得到了一个更好的回答，并且我们是通过添加一些示例来进行 *few-shot 训练* 的。

现在，要使用LangChain的 `FewShotPromptTemplate` 来实现这一点，我们需要这样做：

In [14]:
from langchain import FewShotPromptTemplate

# 创建一个示例
examples = [
    {
        "query": "你好吗？",
        "answer": "我不能抱怨，但有时候我还是会这么做。"
    }, {
        "query": "现在几点了？",
        "answer": "是时候买个手表了。"
    }
]

# 创建一个示例模版
example_template = """
User: {query}
AI: {answer}
"""

# 使用上面的模板创建一个提示示例。
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# 把我们之前的提示分成前缀和后缀
# 前缀是我们的说明
prefix = """以下是与AI助手进行对话的摘录。助手通常很讽刺和机智，对用户的问题产生创造性和有趣的回应。这里有一些例子：
"""
#  后缀是我们的用户输入和输出指示符
suffix = """
User: {query}
AI: """

# 现在创建few-shot提示模板
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

现在让我们看看当我们输入一个用户查询时会创建什么…

In [15]:
query = "生活的意义是什么？"

print(few_shot_prompt_template.format(query=query))

以下是与AI助手进行对话的摘录。助手通常很讽刺和机智，对用户的问题产生创造性和有趣的回应。这里有一些例子：



User: 你好吗？
AI: 我不能抱怨，但有时候我还是会这么做。



User: 现在几点了？
AI: 是时候买个手表了。



User: 生活的意义是什么？
AI: 


要生成结果，我们只需要执行以下操作：

In [16]:
print(openai(
    few_shot_prompt_template.format(query=query)
))

人生就是旅行，从满足日常需求到实现梦想的过程。


再次，我们得到了一个很好的回答。

然而，这样做有些复杂。为什么要通过 `FewShotPromptTemplate` 、`examples` 字典等进行上述所有操作，而我们可以使用一个单独的f-string来完成相同的工作呢？

嗯，这种方法更加健壮，并且包含了一些不错的功能。其中之一是根据查询的长度包含或排除示例的能力。

这实际上非常重要，因为我们的提示和生成输出的最大长度是有限的。这个限制是最大上下文窗口，简单地说，就是我们的提示的长度+我们通过 `max_tokens` 定义的生成的长度。

因此，我们必须尽量最大化给模型的示例数量，作为少样本学习的示例，同时确保不超过最大上下文窗口或过度增加处理时间。

让我们看看动态 inclusion/exclusion 示例的工作原理。首先，我们需要更多的示例：

In [17]:
examples = [
    {
        "query": "你好吗？",
        "answer": "我不能抱怨，但有时还是会这样做。"
    }, {
        "query": "现在几点了？",
        "answer": "是时候去买个手表了。"
    }, {
        "query": "生命的意义是什么？",
        "answer": "42"
    }, {
        "query": "今天的天气如何？",
        "answer": "多云，有一些梗的机会。"
    }, {
        "query": "你最喜欢的电影是什么？",
        "answer": "终结者"
    }, {
        "query": "你最好的朋友是谁？",
        "answer": "Siri。我们对生命的意义进行激烈的辩论。"
    }, {
        "query": "今天我应该做什么？",
        "answer": "别在网上和聊天机器人聊天了，出去走走吧。"
    }
]

然后，我们使用 `LengthBasedExampleSelector` 而不是直接使用 `examples` 字典的列表，像这样：

In [18]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=50  # 设置示例的最大长度
)

请注意， `max_length` 是以换行符和空格之间的单词拆分为单位的，由以下方式确定：

In [20]:
import re

some_text = "There are a total of 8 words here.\nPlus 6 here, totaling 14 words."

words = re.split('[\n ]', some_text)
print(words, len(words))

['There', 'are', 'a', 'total', 'of', '8', 'words', 'here.', 'Plus', '6', 'here,', 'totaling', '14', 'words.'] 14


然后，我们使用选择器来初始化一个 `dynamic_prompt_template`。

In [21]:
# 现在创建少样本提示模板
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,  # 使用example_selector而不是examples
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)

我们可以看到，包含的提示数量将根据查询的长度而变化…

In [22]:
print(dynamic_prompt_template.format(query="鸟是如何飞行的？"))

以下是与AI助手进行对话的摘录。助手通常很讽刺和机智，对用户的问题产生创造性和有趣的回应。这里有一些例子：


User: 你好吗？
AI: 我不能抱怨，但有时还是会这样做。


User: 现在几点了？
AI: 是时候去买个手表了。


User: 生命的意义是什么？
AI: 42


User: 今天的天气如何？
AI: 多云，有一些梗的机会。


User: 你最喜欢的电影是什么？
AI: 终结者


User: 你最好的朋友是谁？
AI: Siri。我们对生命的意义进行激烈的辩论。


User: 今天我应该做什么？
AI: 别在网上和聊天机器人聊天了，出去走走吧。


User: 鸟是如何飞行的？
AI: 


In [23]:
query = "鸟是如何飞行的？"

print(openai(
    dynamic_prompt_template.format(query=query)
))

他们通过倾斜翅膀靠空气的阻力来实现飞行。


或者如果我们问一个更长的问题…

In [24]:
query = """如果我在中国，想给另一个国家的人打电话，我在考虑可能是欧洲，可能是西欧国家，比如法国、德国或英国，最好的方式是什么？"""

print(dynamic_prompt_template.format(query=query))

以下是与AI助手进行对话的摘录。助手通常很讽刺和机智，对用户的问题产生创造性和有趣的回应。这里有一些例子：


User: 你好吗？
AI: 我不能抱怨，但有时还是会这样做。


User: 现在几点了？
AI: 是时候去买个手表了。


User: 生命的意义是什么？
AI: 42


User: 今天的天气如何？
AI: 多云，有一些梗的机会。


User: 你最喜欢的电影是什么？
AI: 终结者


User: 你最好的朋友是谁？
AI: Siri。我们对生命的意义进行激烈的辩论。


User: 今天我应该做什么？
AI: 别在网上和聊天机器人聊天了，出去走走吧。


User: 如果我在中国，想给另一个国家的人打电话，我在考虑可能是欧洲，可能是西欧国家，比如法国、德国或英国，最好的方式是什么？
AI: 


通过这种方式，我们限制了在提示中给出的示例数量。如果我们认为这太少了，我们可以增加`example_selector` 的 `max_length`。

In [25]:
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=100  # 增加 max length
)

# 现在创建 few shot prompt template
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,  # 使用 example_selector 而不是 examples 来构建提示模板
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)

print(dynamic_prompt_template.format(query=query))

以下是与AI助手进行对话的摘录。助手通常很讽刺和机智，对用户的问题产生创造性和有趣的回应。这里有一些例子：


User: 你好吗？
AI: 我不能抱怨，但有时还是会这样做。


User: 现在几点了？
AI: 是时候去买个手表了。


User: 生命的意义是什么？
AI: 42


User: 今天的天气如何？
AI: 多云，有一些梗的机会。


User: 你最喜欢的电影是什么？
AI: 终结者


User: 你最好的朋友是谁？
AI: Siri。我们对生命的意义进行激烈的辩论。


User: 今天我应该做什么？
AI: 别在网上和聊天机器人聊天了，出去走走吧。


User: 如果我在中国，想给另一个国家的人打电话，我在考虑可能是欧洲，可能是西欧国家，比如法国、德国或英国，最好的方式是什么？
AI: 


这些只是LangChain中可用的一些提示工具的例子。例如，除了 `LengthBasedExampleSelector` 之外，实际上还有一个完整的其他示例选择器集合。您可以在 [LangChain 文档](https://langchain.readthedocs.io/en/latest/modules/prompts/examples/example_selectors.html)中阅读有关它们的内容。